# 리뷰의 긍부정을 분류하는 모델을 만들기

## 네이버 영화에서 리뷰를 평점과 함께 크롤링

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
site_pre = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=45290&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='

review=[]
rate=[]
for i in range(1,100):
    site = site_pre + str(i)
    res = requests.get(site)
    soup = BeautifulSoup(res.content,'html.parser')
    
    id_list = []
    id_pre = '_filtered_ment_'
    
    for i in range(10):
        id_list.append(id_pre+str(i))
    
    for id in id_list:
        review.append(soup.find('span',{'id':id}).get_text().strip())
        
    rate_list = []
    rate_list = (soup.select('div.star_score > em'))
    
    for r in rate_list:
        r = int(re.sub('<.+?>', '',str(r)))
        rate.append(r)

In [3]:
df = pd.DataFrame({'review':review,'rate':rate})
df

,review,rate
0,"인셉션은 대단하다 느꼈는데, 인터스텔라는 경이롭다고 느껴진다.",10
1,결론만 말하자면 대박이다 더이상 어떤단어로 칭찬해야하는지도모르겠다.약 3시간의 긴러...,10
2,전율과 환희의 169분이였다. 그 어떤 영화도 시도한 적 없는 명석함과 감동이 담겨...,10
3,이 영화가 명량이나 도둑들보다 관객수가 적다면 진짜 부끄러울듯,10
4,"팝콘, 콜라 사가지 마라.. 먹을시간 없다",10
...,...,...
985,나에게 있어 러닝타임을 30분으로 만들어낸 시각적 예술작품 그 자체이다.,10
986,정말 감동적이고 재밌네요,10
987,최고이 영화를 극장에서 봤다는게 자랑스럽다,10
988,간만에 본 수작이네요 ㄷㄷㄷ,10


In [4]:
pres = ['https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=45290&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=highest&page=',
        'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=45290&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=lowest&page=']

review=[]
rate=[]
target=[]
for pre in pres:
    for i in range(1,200):
        site = pre + str(i)
        res = requests.get(site)
        soup = BeautifulSoup(res.content,'html.parser')
    
        id_list = []
        id_pre = '_filtered_ment_'
    
        for i in range(10):
            id_list.append(id_pre+str(i))
    
        for id in id_list:
            review.append(soup.find('span',{'id':id}).get_text().strip())
        
        rate_list = []
        rate_list = (soup.select('div.star_score > em'))
    
        for r in rate_list:
            r = int(re.sub('<.+?>', '',str(r)))
            rate.append(r)
            if r>8:
                target.append(1)
            elif r<=4:
                target.append(0)
            else:
                target.append(-1)

In [5]:
df2 = pd.DataFrame({'review':review,'rate':rate,'target':target})
df2

,review,rate,target
0,,10,1
1,,10,1
2,내 인생 최고의 영화.딱 이거임,10,1
3,내 인생영화다 진짜 재개봉 했으면…,10,1
4,모든것을 뛰어넘은 명작,10,1
...,...,...,...
3975,난해하고 억지스럽다. 이영화가 왜 이렇게 흥행하는지 이해할 수 없다. 사랑의 힘으로...,3,0
3976,이해력이 낮아 이해못했음,3,0
3977,심야시간에 보았다 + 졸렸다+볼거리 없다 왜케 평점이 높은건지 이해가 안간다(알바들...,3,0
3978,너무 기대를 해서 그런지....별로라는 생각이 드네요...약간 지루했습니다..,3,0


In [6]:
df2.rate.unique()

array([10,  1,  2,  3], dtype=int64)

## 트레이닝/테스트 셋 분리

In [7]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(review, target, test_size=0.2, random_state=0)

In [8]:
len(train_x), len(train_y)

(3184, 3184)

In [9]:
len(test_x), len(test_y)

(796, 796)

- 텍스트 데이터를 정량적인 데이터로 변환해줘야 함

## 학습(적합)과 성능평가

In [10]:
#텍스트 데이터의 벡터화

from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df = 0.9)
tfv.fit(train_x)

tfv_train_x = tfv.transform(train_x)
tfv_train_x

C:\Users\alsgu\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<3184x4900 sparse matrix of type '<class 'numpy.float64'>'
	with 61251 stored elements in Compressed Sparse Row format>

In [11]:
# 로지스틱회귀모형으로 모델 적합

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C':[1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv = 4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [13]:
#테스트 데이터를 통한 예측 정확도 산출
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x,test_y)

0.907035175879397

In [25]:
# 영화리뷰에 대한 긍부정분류
my_review = tfv.transform([input()])
print(type(my_review))
if(grid_cv.best_estimator_.predict(my_review)):
    print(type(my_review))
    print('긍정 리뷰')
else:
    print(type(my_review))
    print('부정 리뷰')

일상의 고단함을 잊게 해주었던 인생영화
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
긍정 리뷰
